# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [5]:
print(spaceship.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None


In [7]:
print(spaceship.describe())

               Age   RoomService     FoodCourt  ShoppingMall           Spa  \
count  8514.000000   8512.000000   8510.000000   8485.000000   8510.000000   
mean     28.827930    224.687617    458.077203    173.729169    311.138778   
std      14.489021    666.717663   1611.489240    604.696458   1136.705535   
min       0.000000      0.000000      0.000000      0.000000      0.000000   
25%      19.000000      0.000000      0.000000      0.000000      0.000000   
50%      27.000000      0.000000      0.000000      0.000000      0.000000   
75%      38.000000     47.000000     76.000000     27.000000     59.000000   
max      79.000000  14327.000000  29813.000000  23492.000000  22408.000000   

             VRDeck  
count   8505.000000  
mean     304.854791  
std     1145.717189  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%       46.000000  
max    24133.000000  


In [9]:
null_counts = spaceship.isnull().sum()
print(null_counts[null_counts > 0])

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
dtype: int64


In [11]:
spaceship['Age'] = spaceship['Age'].fillna(spaceship['Age'].median())

In [13]:
spending_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
spaceship[spending_columns] = spaceship[spending_columns].fillna(0)

In [15]:
cat_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']

cat_imputer = SimpleImputer(strategy='most_frequent')
spaceship[cat_cols] = cat_imputer.fit_transform(spaceship[cat_cols])

In [17]:
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [19]:
null_counts = spaceship.isnull().sum()
print(null_counts[null_counts > 0])

Series([], dtype: int64)


In [21]:
spaceship['Cabin'].sort_values(ascending = True).unique()

array(['A/0/P', 'A/0/S', 'A/1/S', ..., 'T/2/P', 'T/2/S', 'T/3/P'],
      dtype=object)

In [23]:
spaceship['CabinLabel'] = spaceship['Cabin'].str[0]

In [25]:
spaceship['CabinLabel'].sort_values(ascending = True).unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'], dtype=object)

In [27]:
spaceship_cleaned = spaceship.drop(columns=['PassengerId', 'Name', 'Cabin'])

spaceship_cleaned

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,CabinLabel
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E


In [29]:
spaceship_cleaned[['VIP', 'CryoSleep', 'Transported']] = spaceship_cleaned[['VIP', 'CryoSleep', 'Transported']].astype(int)

In [31]:
numerical_features = spaceship_cleaned.select_dtypes(include=['float64', 'int64'])
numerical_features.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1


In [33]:
categorical_columns = ['HomePlanet', 'Destination', 'CabinLabel']
spaceship_cleaned = pd.get_dummies(spaceship_cleaned, columns=categorical_columns, drop_first=True)

spaceship_cleaned

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,CabinLabel_B,CabinLabel_C,CabinLabel_D,CabinLabel_E,CabinLabel_F,CabinLabel_G,CabinLabel_T
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,True,False,False,True,True,False,False,False,False,False,False
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,False,False,False,True,False,False,False,False,True,False,False
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,True,False,False,True,False,False,False,False,False,False,False
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,True,False,False,True,False,False,False,False,False,False,False
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,False,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,True,False,False,False,False,False,False,False,False,False,False
8689,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0,False,False,True,False,False,False,False,False,False,True,False
8690,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,False,False,False,True,False,False,False,False,False,True,False
8691,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0,True,False,False,False,False,False,False,True,False,False,False


In [35]:
features = spaceship_cleaned.drop(columns = numerical_features, axis=1) #X
target = spaceship_cleaned['Transported'] #y
spaceship_cleaned

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,CabinLabel_B,CabinLabel_C,CabinLabel_D,CabinLabel_E,CabinLabel_F,CabinLabel_G,CabinLabel_T
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,True,False,False,True,True,False,False,False,False,False,False
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,False,False,False,True,False,False,False,False,True,False,False
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,True,False,False,True,False,False,False,False,False,False,False
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,True,False,False,True,False,False,False,False,False,False,False
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,False,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,True,False,False,False,False,False,False,False,False,False,False
8689,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0,False,False,True,False,False,False,False,False,False,True,False
8690,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,False,False,False,True,False,False,False,False,False,True,False
8691,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0,True,False,False,False,False,False,False,True,False,False,False


In [37]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [39]:
#normalizer = MinMaxScaler()
normalizer = StandardScaler()
normalizer.fit(X_train)

StandardScaler()

In [41]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [43]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [ ]:
# ---> AdaBoost and Bagging/Pasting are based on tree-based algorithms (e.g., Decision Trees).
# Tree-based models are scale-invariant, meaning they don't rely on feature scaling 
# or normalization since they split data based on feature thresholds.
# Thus, you don't need to apply StandardScaler or MinMaxScaler when using these models.

In [45]:
bagging_reg_scaled = BaggingRegressor(
    n_estimators=100,
    max_samples=0.8,
    max_features=0.8,
    random_state=42
)

In [47]:
bagging_reg_scaled.fit(X_train_norm, y_train)

BaggingRegressor(max_features=0.8, max_samples=0.8, n_estimators=100,
                 random_state=42)

In [49]:
pred = bagging_reg_scaled.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", bagging_reg_scaled.score(X_test_norm, y_test))

MAE 0.4600996947591204
RMSE 0.47620142260169573
R2 score 0.09287812684311536


In [51]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=50)

In [53]:
ada_reg.fit(X_train_norm, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20))

In [55]:
pred = ada_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test_norm, y_test))

MAE 0.46397571122180553
RMSE 0.476797892278902
R2 score 0.09060425899536928


- Evaluate your model

In [1]:
# Best Overall Model - AdaBoosting, based on its lowest MAE 
# (better error performance for prediction). But, it has a moderate R² score.
# It's best for minimizing Prediction Error (MAE/RMSE). 
# Best for Variance Explanation - Bagging and Pasting (highest R²). Best for 
# Explaining Variability (R²). 

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [60]:
grid = {"n_estimators": [50, 100, 150],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

- Run Grid Search

In [62]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())

In [64]:
model = GridSearchCV(estimator = ada_reg, param_grid = grid, cv=5)
#model = GridSearchCV(estimator=ada_reg, param_grid, cv=5, scoring='r2')

In [66]:
model.fit(X_train_norm, y_train)

GridSearchCV(cv=5,
             estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor()),
             param_grid={'estimator__max_depth': [10, 30, 50],
                         'estimator__max_leaf_nodes': [250, 500, 1000, None],
                         'n_estimators': [50, 100, 150]})

In [68]:
print("Model has been fitted.")

Model has been fitted.


In [71]:
model.best_params_

{'estimator__max_depth': 30,
 'estimator__max_leaf_nodes': None,
 'n_estimators': 50}

In [73]:
best_model = model.best_estimator_

In [75]:
pred = ada_reg.predict(X_test_norm)

NotFittedError: This AdaBoostRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [77]:
print(X_train_norm.shape)
print(y_train.shape)

(6954, 11)
(6954,)


In [82]:
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test_norm, y_test))

MAE 0.46397571122180553
RMSE 0.476797892278902


NotFittedError: This AdaBoostRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

- Evaluate your model